# OGR

## THIS NOTEBOOK IS UNDER CONSTRUCTION AND NOT USED IN CURRENT VERSION OF THE WORKSHOP


[OGR](http://gdal.org/ogr) is part of [GDAL](http://gdal.org/) library. It is traditional way how to interact with vector data. Currently it supports about 80 vector formats.

OGR API overpasses differences between various vector formats, services, database etc.:

* **Driver** - driver for reading and writing the data to specified format
* **Data Source** - whatever source (file, database, service, ...)
* **Layer** - data layer within the source (file content, database table, ...)
* **Feature** - vector feature
* **Field, Geometry** - attributes and geomtry

![OGR API](http://training.gismentors.eu/geopython/_images/aafig-2532774d3eb3b9d2cd66862013ca5a161fd76217.svg "OGR API")


OGR-Python interface is abstract API on top of original classes and methods of original C++ code. Also because of this, some approaches seem complicated, compared native Python code, like e.g. Fiona.

Documentation: http://www.gdal.org/ogr_apitut.html

API: http://gdal.org/python/

## Buffer

First we need to open *data source*

In [2]:
from osgeo import ogr
ds = ogr.Open("../../data/europe_110.geo.json")
print(ds)
print(ds.GetLayerCount())

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f28646e3f90> >
1


Next we have to open *layer* (for files, there is usually no reason for separate layer within data source, but for example for database data source, a layer is reference to concrete table).

In [3]:
l = ds.GetLayer(0)
print(l)
print(l.GetFeatureCount())

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x7f28432d89f0> >
39


Schema of the layer, definition of geometry type and attribution fields:

In [4]:
l.GetGeomType()

0

In [5]:
l.GetGeomType() == ogr.wkbPolygon

False

In [6]:
l.schema

[<osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8a50> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8960> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8bd0> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8c00> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8c30> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8c60> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8c90> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8cc0> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8cf0> >,
 <osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x7f28432d8d20> >,
 <osgeo.ogr.FieldDef

In [7]:
l.schema[4].name

'sov_a3'

Print NAZEV attribute of all features

In [8]:
features_nr = l.GetFeatureCount()
for i in range(features_nr):
    f = l.GetNextFeature()
    print f.GetField('NAZEV')

SyntaxError: invalid syntax (<ipython-input-8-1c7fae15bba7>, line 4)

Get vector feature bounding box (envelope):

In [9]:
f = l.GetFeature(54)
geom = f.GetGeometryRef()
geom.GetEnvelope()

AttributeError: 'NoneType' object has no attribute 'GetGeometryRef'

Get geometry centroid

In [ ]:
c = geom.Centroid()
c.GetPoint()

Get geometry buffer

In [ ]:
buff = c.Buffer(100)
geom.Intersects(buff)

## Complete example

In this example we will demonstrate work with vector data from begining to the end: open data set, metadata, attribute change, saving of new attribute back to the file. With Fiona, this would be about 3x simplier. However, OGR accesses the data on much lower level compared to Fiona, therefore bigger datasets can be interfaced.

In [ ]:
from osgeo import osr

# Creating new file with new driver
drv = ogr.GetDriverByName('GML')
ds = drv.CreateDataSource('../../data/out.gml')
srs = osr.SpatialReference()
srs.ImportFromEPSG(5514)
print(srs.ExportToProj4())
layer = ds.CreateLayer('out.gml', srs, ogr.wkbLineString)

# create new attributes named and code
field_name = ogr.FieldDefn('name', ogr.OFTString)
field_name.SetWidth(24)
field_number = ogr.FieldDefn('code', ogr.OFTInteger)
layer.CreateField(field_name)
layer.CreateField(field_number)

# create new line geometry and read from WKT
line = ogr.CreateGeometryFromWkt('LINESTRING(%f %f, %f %f)' % (0, 0, 1, 1))

# create new feature, set attributes and geometry
feature = ogr.Feature(layer.GetLayerDefn())
feature.SetGeometry(line)
feature.SetField("name", 'the line')
feature.SetField("code", 42)

layer.CreateFeature(feature)

# final cleaning
feature.Destroy()
ds.Destroy()

now we can check the result

In [ ]:
ds = ogr.Open('../../data/out.gml')
layer = ds.GetLayer(0)
print(layer.GetFeatureCount())
print(layer.GetFeature(0).GetField('name'))
ds.Destroy()